In [150]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from datetime import datetime,date,timezone

In [151]:
customer_df=pd.read_csv("https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/refs/heads/main/Section-13.1-Feature-Store/raw/customers.csv")

In [152]:
customer_df.head()

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,brooke williams,F,alabama,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z
1,C2,jim reese,M,oregon,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z
2,C3,adam walker phd,M,north dakota,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z
3,C4,nathan roberts,M,louisiana,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z
4,C5,richard adkins,M,idaho,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z


In [153]:
label_encoder=LabelEncoder()
min_max_scaler=MinMaxScaler()

In [154]:
customer_df.drop(columns=["name","state"],axis=1,inplace=True)

In [155]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   10000 non-null  object
 1   sex           10000 non-null  object
 2   age           10000 non-null  int64 
 3   is_married    10000 non-null  bool  
 4   active_since  10000 non-null  object
 5   event_time    10000 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 400.5+ KB


In [156]:
bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
customer_df['age_range'] = pd.cut(customer_df['age'], bins, labels=labels, include_lowest=True)

In [157]:
customer_df.head()

,customer_id,sex,age,is_married,active_since,event_time,age_range
0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus
1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus
2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39
3,C4,M,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,60-69
4,C5,M,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,60-69


In [158]:
df_age_group = pd.get_dummies(customer_df['age_range'],prefix="age", dtype='int')

In [159]:
df_age_group.head()

,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,1,0,0,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


In [160]:
customers_df = pd.concat([customer_df,df_age_group],axis=1)
customers_df.head()

,customer_id,sex,age,is_married,active_since,event_time,age_range,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus,0,0,0,0,0,1
1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus,0,0,0,0,0,1
2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39,0,1,0,0,0,0
3,C4,M,68,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,60-69,0,0,0,0,1,0
4,C5,M,62,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,60-69,0,0,0,0,1,0


In [161]:
customers_df.drop(columns=['age','age_range'], inplace=True,axis=1)
customers_df['sex'] = customers_df['sex'].map({"F":0,"M":1})

In [162]:
customers_df.head()

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,0,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,C2,1,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,C3,1,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,C4,1,False,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,C5,1,False,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [163]:

customers_df['is_married'] = customers_df['is_married'].astype('int')
customers_df['active_since'] = pd.to_datetime(customers_df['active_since'], format="%Y-%m-%d %H:%M:%S")

In [164]:
customers_df.head()

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,0,1,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,C2,1,1,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,C3,1,1,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,C4,1,0,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,C5,1,0,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [165]:
def get_delta_days(datetime) -> int :
    today = date.today()
    delta = today - datetime.date()
    return delta.days

customers_df['n_days_active'] = customers_df['active_since'].apply(lambda x: get_delta_days(x))
customers_df['n_days_active'] = min_max_scaler.fit_transform(customers_df[['n_days_active']])
customers_df.drop("active_since",axis=1, inplace=True)

In [166]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [167]:
customers_df.to_csv("customers.csv")

In [168]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


# Transform Orders Data

In [169]:
orders_df = pd.read_csv("https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/main/Section-13.1-Feature-Store/raw/orders.csv")

In [170]:
orders_df.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z
1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z
2,O3,C2139,P4749,57.17,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z
3,O4,C7794,P542,5.35,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z
4,O5,C2229,P7605,47.85,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z


In [171]:
orders_df['purchased_on'] =  pd.to_datetime(orders_df['purchased_on'], format='%Y-%m-%d %H:%M:%S')
orders_df['n_days_since_last_purchase'] = orders_df['purchased_on'].apply(lambda x: get_delta_days(x))

In [172]:
orders_df['n_days_since_last_purchase'] = min_max_scaler.fit_transform(orders_df[['n_days_since_last_purchase']])
orders_df.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,57.17,1,2020-11-01 22:09:22,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,5.35,1,2020-03-22 10:10:38,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,47.85,1,2021-01-14 01:08:56,2024-05-02T05:39:17.172Z,0.265504


In [173]:
orders_df['purchase_amount'] = min_max_scaler.fit_transform(orders_df[['purchase_amount']])

In [174]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   order_id                    100000 non-null  object        
 1   customer_id                 100000 non-null  object        
 2   product_id                  100000 non-null  object        
 3   purchase_amount             100000 non-null  float64       
 4   is_reordered                100000 non-null  int64         
 5   purchased_on                100000 non-null  datetime64[ns]
 6   event_time                  100000 non-null  object        
 7   n_days_since_last_purchase  100000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 6.1+ MB


In [175]:
orders_df['is_reordered'].value_counts()

is_reordered
1    66509
0    33491
Name: count, dtype: int64

In [176]:
orders_df.drop(columns="purchased_on", inplace=True)

In [177]:
orders_df.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,0.572673,0,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,0.693861,0,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,0.556139,1,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,0.043069,1,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,0.463861,1,2024-05-02T05:39:17.172Z,0.265504


In [178]:
orders_df.to_csv("orders.csv", index=False)